## Imports

In [17]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from statsmodels.iolib.summary2 import summary_col
from utils.formatting import normalize_string, pascal_to_snake, get_formula

## Loading Files

In [18]:
df_telco = pd.read_csv("datasets/telco-customer-churn.csv")

## Data Wrangling

In [19]:
pd.set_option("display.max.columns", None)

In [20]:
# Visualize Data
df_telco

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,6840-RESVB,Male,0,Yes,Yes,24,Yes,Yes,DSL,Yes,No,Yes,Yes,Yes,Yes,One year,Yes,Mailed check,84.80,1990.5,No
7039,2234-XADUH,Female,0,Yes,Yes,72,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,One year,Yes,Credit card (automatic),103.20,7362.9,No
7040,4801-JZAZL,Female,0,Yes,Yes,11,No,No phone service,DSL,Yes,No,No,No,No,No,Month-to-month,Yes,Electronic check,29.60,346.45,No
7041,8361-LTMKD,Male,1,Yes,No,4,Yes,Yes,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Mailed check,74.40,306.6,Yes


In [21]:
# Inspecting DataFrame structure
df_telco.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [22]:
# Summary numeric variables
df_telco.describe()

,SeniorCitizen,tenure,MonthlyCharges
count,7043.000000,7043.000000,7043.000000
mean,0.162147,32.371149,64.761692
std,0.368612,24.559481,30.090047
min,0.000000,0.000000,18.250000
25%,0.000000,9.000000,35.500000
50%,0.000000,29.000000,70.350000
75%,0.000000,55.000000,89.850000
max,1.000000,72.000000,118.750000


In [23]:
# Summary non-numeric variables
df_telco.select_dtypes(include=object).describe().T

,count,unique,top,freq
customerID,7043,7043,3186-AJIEK,1
gender,7043,2,Male,3555
Partner,7043,2,No,3641
Dependents,7043,2,No,4933
PhoneService,7043,2,Yes,6361
MultipleLines,7043,3,No,3390
InternetService,7043,3,Fiber optic,3096
OnlineSecurity,7043,3,No,3498
OnlineBackup,7043,3,No,3088
DeviceProtection,7043,3,No,3095


In [24]:
df_telco.rename({
    "customerID": "customer_id",
    "StreamingTV": "streaming_tv"
    },
axis=1,
inplace=True
)

df_telco.columns = pascal_to_snake(df_telco.columns)

df_telco.columns

Index(['customer_id', 'gender', 'senior_citizen', 'partner', 'dependents',
       'tenure', 'phone_service', 'multiple_lines', 'internet_service',
       'online_security', 'online_backup', 'device_protection', 'tech_support',
       'streaming_tv', 'streaming_movies', 'contract', 'paperless_billing',
       'payment_method', 'monthly_charges', 'total_charges', 'churn'],
      dtype='object')

In [25]:
# Fix total_charges data type
df_telco["total_charges"] = pd.to_numeric(df_telco.total_charges, errors='coerce')

# Checking for missing values (NA) in rows
na_count = df_telco.isna().any(axis=1).sum()
print(f"Number of rows with missing values: {na_count}")


Number of rows with missing values: 11


In [26]:
# Droping NA rows
df_telco.dropna(how='any', inplace=True)

# Checking for missing values (NA) in rows
na_count = df_telco.isna().any(axis=1).sum()
print(f"Number of rows with missing values: {na_count}")

Number of rows with missing values: 0


In [27]:
# One-hot encoding
df_telco_dummies = pd.get_dummies(
    data=df_telco,
    columns=[
        "gender",
        "partner",
        "dependents",
        "phone_service",
        "multiple_lines",
        "internet_service",
        "online_security",
        "device_protection",
        "tech_support",
        "online_backup",
        "streaming_tv",
        "streaming_movies",
        "contract",
        "paperless_billing",
        "payment_method",
    ],
    drop_first=True,
    dtype=int
)

df_telco_dummies["churn"] = np.where(df_telco_dummies.churn == 'Yes', 1, 0)

df_telco_dummies.columns = normalize_string(df_telco_dummies.columns.str.replace("[()]", "", regex=True))

# Visualize data with dummies
df_telco_dummies

,customer_id,senior_citizen,tenure,monthly_charges,total_charges,churn,gender_male,partner_yes,dependents_yes,phone_service_yes,multiple_lines_no_phone_service,multiple_lines_yes,internet_service_fiber_optic,internet_service_no,online_security_no_internet_service,online_security_yes,device_protection_no_internet_service,device_protection_yes,tech_support_no_internet_service,tech_support_yes,online_backup_no_internet_service,online_backup_yes,streaming_tv_no_internet_service,streaming_tv_yes,streaming_movies_no_internet_service,streaming_movies_yes,contract_one_year,contract_two_year,paperless_billing_yes,payment_method_credit_card_automatic,payment_method_electronic_check,payment_method_mailed_check
0,7590-VHVEG,0,1,29.85,29.85,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0
1,5575-GNVDE,0,34,56.95,1889.50,0,1,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1
2,3668-QPYBK,0,2,53.85,108.15,1,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1
3,7795-CFOCW,0,45,42.30,1840.75,0,1,0,0,0,1,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0
4,9237-HQITU,0,2,70.70,151.65,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,6840-RESVB,0,24,84.80,1990.50,0,1,1,1,1,0,1,0,0,0,1,0,1,0,1,0,0,0,1,0,1,1,0,1,0,0,1
7039,2234-XADUH,0,72,103.20,7362.90,0,0,1,1,1,0,1,1,0,0,0,0,1,0,0,0,1,0,1,0,1,1,0,1,1,0,0
7040,4801-JZAZL,0,11,29.60,346.45,0,0,1,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0
7041,8361-LTMKD,1,4,74.40,306.60,1,1,1,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1


In [28]:
df_telco_dummies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7032 entries, 0 to 7042
Data columns (total 32 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   customer_id                            7032 non-null   object 
 1   senior_citizen                         7032 non-null   int64  
 2   tenure                                 7032 non-null   int64  
 3   monthly_charges                        7032 non-null   float64
 4   total_charges                          7032 non-null   float64
 5   churn                                  7032 non-null   int64  
 6   gender_male                            7032 non-null   int64  
 7   partner_yes                            7032 non-null   int64  
 8   dependents_yes                         7032 non-null   int64  
 9   phone_service_yes                      7032 non-null   int64  
 10  multiple_lines_no_phone_service        7032 non-null   int64  
 11  multiple_

## Logistic Model

In [29]:
# Model Formula
formula = get_formula(df_telco_dummies, endog='churn', drop=["customer_id"])

print(formula)

churn ~ senior_citizen + tenure + monthly_charges + total_charges + gender_male + partner_yes + dependents_yes + phone_service_yes + multiple_lines_no_phone_service + multiple_lines_yes + internet_service_fiber_optic + internet_service_no + online_security_no_internet_service + online_security_yes + device_protection_no_internet_service + device_protection_yes + tech_support_no_internet_service + tech_support_yes + online_backup_no_internet_service + online_backup_yes + streaming_tv_no_internet_service + streaming_tv_yes + streaming_movies_no_internet_service + streaming_movies_yes + contract_one_year + contract_two_year + paperless_billing_yes + payment_method_credit_card_automatic + payment_method_electronic_check + payment_method_mailed_check


In [30]:
logit_model = smf.logit(data=df_telco_dummies, formula=formula).fit()

Optimization terminated successfully.
         Current function value: 0.414269
         Iterations 9


In [31]:
logit_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  churn   No. Observations:                 7032
Model:                          Logit   Df Residuals:                     7008
Method:                           MLE   Df Model:                           23
Date:                Wed, 27 Nov 2024   Pseudo R-squ.:                  0.2845
Time:                        20:17:15   Log-Likelihood:                -2913.1
converged:                       True   LL-Null:                       -4071.7
Covariance Type:            nonrobust   LLR p-value:                     0.000
=========================================================================================================
                                            coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------
Intercept                                 0.8340        nan        nan        nan         nan         nan
senior_citizen                            0.2168      0.057      3.777      0.000       0.104       0.329
tenure                                   -0.0606      0.007     -9.022      0.000      -0.074      -0.047
monthly_charges                          -0.0403      0.008     -4.798      0.000      -0.057      -0.024
total_charges                             0.0003   7.33e-05      4.488      0.000       0.000       0.000
gender_male                              -0.0218      0.065     -0.337      0.736      -0.149       0.105
partner_yes                              -0.0004      0.078     -0.005      0.996      -0.153       0.152
dependents_yes                           -0.1485      0.089     -1.661      0.097      -0.324       0.027
phone_service_yes                         0.5027        nan        nan        nan         nan         nan
multiple_lines_no_phone_service           0.3313        nan        nan        nan         nan         nan
multiple_lines_yes                        0.4484      0.090      5.008      0.000       0.273       0.624
internet_service_fiber_optic              1.7475      0.201      8.709      0.000       1.354       2.141
internet_service_no                      -0.2552   3.91e+14  -6.53e-16      1.000   -7.66e+14    7.66e+14
online_security_no_internet_service      -0.2552   1.17e+07  -2.19e-08      1.000   -2.29e+07    2.29e+07
online_security_yes                      -0.2054      0.089     -2.310      0.021      -0.380      -0.031
device_protection_no_internet_service    -0.2552   1.98e+15  -1.29e-16      1.000   -3.89e+15    3.89e+15
device_protection_yes                     0.1474      0.086      1.708      0.088      -0.022       0.316
tech_support_no_internet_service         -0.2552   1.19e+07  -2.15e-08      1.000   -2.32e+07    2.32e+07
tech_support_yes                         -0.1805      0.096     -1.886      0.059      -0.368       0.007
online_backup_no_internet_service        -0.2552   1.95e+07  -1.31e-08      1.000   -3.83e+07    3.83e+07
online_backup_yes                         0.0260      0.087      0.301      0.763      -0.144       0.196
streaming_tv_no_internet_service         -0.2552        nan        nan        nan         nan         nan
streaming_tv_yes                          0.5905      0.127      4.653      0.000       0.342       0.839
streaming_movies_no_internet_service     -0.2552        nan        nan        nan         nan         nan
streaming_movies_yes                      0.5993      0.038     15.978      0.000       0.526       0.673
contract_one_year                        -0.6608      0.091     -7.230      0.000      -0.840      -0.482
contract_two_year                        -1.3571      0.170     -7.996      0.000      -1.690      -1.024
paperless_billing_yes                     0.3424      0.080  